In [ ]:
from tensorflow import keras
from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
import sys

import numpy as np


# Generator搭建：
1. G中的输入可以采用全连接层但是需要reshape成一个4 dimension的tensor(Batch_size, Height, Width, Channel)
2. pooling层改成卷积层，G用的fractional-strided convolutions，就是逆卷积，需要上采样
3. 需要BatchNormalization，但是不对G的output layer使用BN
4. G的激活函数用的ReLU，output层用的Tanh；


In [ ]:
# build_generator(self)
model = Sequential()

model.add(Dense(128 * 8 * 8, activation="relu", input_dim=100)) # 8,8,128
model.add(Reshape((8, 8, 128)))

model.add(UpSampling2D()) # 16,16,128

model.add(Conv2D(128, kernel_size=3, padding="same"))
model.add(BatchNormalization(momentum=0.8))
model.add(Activation("relu"))

model.add(UpSampling2D()) # 32,32,128

model.add(Conv2D(64, kernel_size=3, padding="same")) 
model.add(BatchNormalization(momentum=0.8))
model.add(Activation("relu"))

model.add(Conv2D(3, kernel_size=3, padding="same"))
model.add(Activation("tanh"))

model.summary()

noise = Input(shape=(100,))
img = model(noise)        
        
generator = Model(noise, img)


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 8192)              827392    
_________________________________________________________________
reshape_3 (Reshape)          (None, 8, 8, 128)         0         
_________________________________________________________________
up_sampling2d_5 (UpSampling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 128)       147584    
_________________________________________________________________
batch_normalization_11 (Batc (None, 16, 16, 128)       512       
_________________________________________________________________
activation_7 (Activation)    (None, 16, 16, 128)       0         
_________________________________________________________________
up_sampling2d_6 (UpSampling2 (None, 32, 32, 128)      

# Discriminator搭建：
1. D中用全卷积层代替池化层，strided convolutions
2. 取消全连接层：D中最后的卷积层可以先flatten然后送入一个sigmoid分类器
3. 用batchnorm，但是不对D的input layer使用BN
4. 在D中所有层的激活函数都是用LeakyReLU

In [ ]:
# build_discriminator
model = Sequential()

model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=(32,32,3), padding="same")) # 16,16,32
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))  # 8,8,64
model.add(BatchNormalization(momentum=0.8))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=3, strides=2, padding="same")) # 4,4,128
model.add(BatchNormalization(momentum=0.8))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=3, strides=1, padding="same")) # 4,4,256
model.add(BatchNormalization(momentum=0.8))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.25))

model.add(Flatten()) # 4*4*256
model.add(Dense(1, activation='sigmoid'))

model.summary()

img = Input(shape=(32,32,3)) # 输入 （28，28，1）
validity = model(img) # 输出二分类结果

discriminator = Model(img, validity)


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 8, 8, 64)          18496     
_________________________________________________________________
batch_normalization_13 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 8, 8, 64)         

优化器和论文上的不同，没有用Adam而是RMSprop，这样效果会更好

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)

# discriminator
discriminator.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])


# The combined model  (stacked generator and discriminator)
z = Input(shape=(100,))
img = generator(z)
validity = discriminator(img)
# For the combined model we will only train the generator
discriminator.trainable = False

# Trains the generator to fool the discriminator
combined = Model(z, validity)
combined.summary()
combined.compile(loss='binary_crossentropy', 
                 optimizer=optimizer)
 

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 100)               0         
_________________________________________________________________
model_7 (Model)              (None, 32, 32, 3)         1051267   
_________________________________________________________________
model_8 (Model)              (None, 1)                 394305    
Total params: 1,445,572
Trainable params: 1,050,883
Non-trainable params: 394,689
_________________________________________________________________


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
def sample_images(epoch):
    r, c = 5, 10
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = (0.5 * gen_imgs + 0.5)

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,:])# 注意这里的改变
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("/content/drive/My Drive/images0/%d.png" % epoch)
    plt.close()


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# import os
# from google.colab import drive
# drive.mount('/content/drive')

# path = "/content/drive/My Drive/cifar-10-python.tar.gz"


把数据集的10个不同标签的图片放到一个10*n维的list arrX里面

In [ ]:
batch_size = 32
sample_interval = 2000
# Load the dataset
(X_train,Label_train),(X_test,Label_test)=cifar10.load_data()#(50000, 32, 32, 3)
print("train data:",'images:',X_train.shape,
      " labels:",Label_train.shape)
# 归一化 Rescale -1 to 1
X_train = X_train / 127.5 - 1.

arrX=[[] for i in range(10)]
for i in range(len(Label_train)):
  if Label_train[i]==0:
    arrX[0].append(X_train[i])
  elif Label_train[i]==1:
    arrX[1].append(X_train[i])
  elif Label_train[i]==2:
    arrX[2].append(X_train[i])
  elif Label_train[i]==3:
    arrX[3].append(X_train[i])
  elif Label_train[i]==4:
    arrX[4].append(X_train[i])
  elif Label_train[i]==5:
    arrX[5].append(X_train[i])
  elif Label_train[i]==6:
    arrX[6].append(X_train[i])
  elif Label_train[i]==7:
    arrX[7].append(X_train[i])
  elif Label_train[i]==8:
    arrX[8].append(X_train[i])
  elif Label_train[i]==9:
    arrX[9].append(X_train[i])
arrX=np.array(arrX)
# Adversarial ground truths
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)


训练不同标签的图片需要改变image_type的值即可；


In [ ]:

for epoch in range(50001):
    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random batch of images
    image_type=1
    idx = np.random.randint(0, arrX[image_type].shape[0], batch_size) # 随机抽  
    imgs = arrX[image_type][idx]
    noise = np.random.normal(0, 1, (batch_size, 100))# 生成标准的高斯分布噪声

    # Generate a batch of new images
    gen_imgs = generator.predict(noise)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(imgs, valid) #真实数据对应标签１
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake) #生成的数据对应标签０
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator
    # ---------------------
    noise = np.random.normal(0, 1, (batch_size, 100))

    # Train the generator (to have the discriminator label samples as valid)
    g_loss = combined.train_on_batch(noise, valid)

    # Plot the progress
    if epoch % sample_interval==0:
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    # If at save interval => save generated image samples
    if epoch % sample_interval == 0:
        sample_images(epoch)
        


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 5.418295, acc.: 23.44%] [G loss: 0.575614]
500 [D loss: 0.854747, acc.: 23.44%] [G loss: 0.856658]
1000 [D loss: 0.779795, acc.: 37.50%] [G loss: 0.754063]
1500 [D loss: 0.812357, acc.: 43.75%] [G loss: 0.767120]
2000 [D loss: 0.647590, acc.: 71.88%] [G loss: 0.692581]
2500 [D loss: 0.641821, acc.: 79.69%] [G loss: 0.877559]
3000 [D loss: 0.713240, acc.: 40.62%] [G loss: 0.708175]
3500 [D loss: 0.789789, acc.: 42.19%] [G loss: 0.746944]
4000 [D loss: 0.706017, acc.: 39.06%] [G loss: 0.752689]
4500 [D loss: 0.689177, acc.: 64.06%] [G loss: 0.737503]
5000 [D loss: 0.686403, acc.: 53.12%] [G loss: 0.791514]
5500 [D loss: 0.683004, acc.: 56.25%] [G loss: 0.766238]
6000 [D loss: 0.708228, acc.: 48.44%] [G loss: 0.805275]
6500 [D loss: 0.675979, acc.: 56.25%] [G loss: 0.771018]
7000 [D loss: 0.708185, acc.: 57.81%] [G loss: 0.641927]
7500 [D loss: 0.737197, acc.: 48.44%] [G loss: 0.689338]
8000 [D loss: 0.700146, acc.: 59.38%] [G loss: 0.712323]
8500 [D loss: 0.767745, acc.: 53.12